In [3]:
import os
# 【硬性要求】加载环境变量 (尽管单纯加载 PDF 暂不需要大模型 API，但保持标准起手式是好习惯)
from dotenv import load_dotenv
load_dotenv(override=True)

# 推荐写法（目前最稳妥）
from langchain_community.document_loaders import PyPDFLoader

# ==========================================
# 1. 准备阶段 (模拟一个 PDF 文件)
# ==========================================
# 为了保证代码在你本地绝对可运行，如果你没有 handbook.pdf，
# 这里有一个简单的一键生成测试 PDF 的占位逻辑。
# (注意：运行此块需要先安装: pip install fpdf)
PDF_PATH = "data/handbook.pdf"
os.makedirs("data", exist_ok=True)
if not os.path.exists(PDF_PATH):
    print(f"[提示] 未找到 {PDF_PATH}，正在为你生成一个测试用的 3 页 PDF...")
    try:
        from fpdf import FPDF
        pdf = FPDF()
        for i in range(1, 4):
            pdf.add_page()
            pdf.set_font("Arial", size=12)
            pdf.cell(200, 10, txt=f"This is the content of Page {i} in Lakeside Bicycles handbook.", ln=True)
        pdf.output(PDF_PATH)
    except ImportError:
        print("[警告] 请使用 'pip install fpdf' 安装库以生成测试 PDF，或者自行放一个 pdf 到 data 目录下。")

# ==========================================
# 2. 实例化加载器并执行 ETL
# ==========================================
try:
    # 实例化加载器，指向目标文件
    loader = PyPDFLoader(PDF_PATH)
    
    # load_and_split() 会在加载的同时按页进行物理分割
    print("开始加载并分割 PDF...")
    pages = loader.load_and_split()
    
    print(f"\n✅ 成功加载！文档被分割成了 {len(pages)} 个 Document 对象。\n")
    
    # ==========================================
    # 3. 深入探究 Document 对象内部 (科研级重点)
    # ==========================================
    # 我们取出第二页 (索引为 1) 来看看里面到底有什么
    if len(pages) > 1:
        doc = pages[1]
        
        print("=== 页面正文 (page_content) ===")
        # .strip() 去除可能存在的首尾空白符，这在清理真实杂乱数据时很常用
        print(doc.page_content.strip()) 
        print("-" * 30)
        
        print("=== 元数据 (metadata) ===")
        # 你会看到类似 {'source': 'data/handbook.pdf', 'page': 1} 的信息
        print(doc.metadata) 
        
except FileNotFoundError:
    print(f"❌ 错误：找不到文件 {PDF_PATH}。请确保文件路径正确。")

开始加载并分割 PDF...

✅ 成功加载！文档被分割成了 3 个 Document 对象。

=== 页面正文 (page_content) ===
While technical models provide the "how" of security, they fail to address the
ethical "why." The integration of advanced surveillance and Artificial Intelligence (AI)
has exacerbated the tension between security and civil liberties. A poignant example
is the use of facial recognition technology. While companies like Clearview AI argue
that scraping billions of public images helps law enforcement solve crimes, privacy
advocates contend that this constitutes a violation of anonymity (Hill, 2020). As
Manjikian (2017) articulates, this scenario presents a classic conflict between
utilitarian ethics, which justifies sacrificing individual privacy for the perceived
greater good of collective security, and deontological perspectives, which view
privacy as an inviolable duty that cannot be traded away.
Furthermore, the reliance on AI for security decision-making introduces
critical questions of accountability. This i